## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# nvda 134.97 1000

In [4]:
symbol = 'AAPL'.upper()

# 5 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '5m', skip_dl=False) # skip_dl=True on redo's
print(f'{symbol} 5m Interval Timestamp: {time_stamp} ')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 15 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '15m', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 15m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 30 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '30m', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 30m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Hour Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1h', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1h Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Day Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1d', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1d Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Week Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1wk', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1wk Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

AAPL 5m Interval Timestamp: 2024-10-13 03:07:03 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,1.0,0.947432,0.947217,0.703468
prob_up,0.0,0.021995,0.009964,0.086369
prob_static,1.0,0.962452,0.962298,0.788192
prob_down,0.0,0.015554,0.027738,0.12544
precision,0.675147,0.664,0.654902,0.583618
recall,0.721757,0.694561,0.698745,0.715481
f1,0.697674,0.678937,0.676113,0.642857
support,"[479.0, 478.0, 477.0]","[479.0, 478.0, 477.0]","[479.0, 478.0, 477.0]","[479.0, 478.0, 477.0]"



AAPL 15m Interval Timestamp: 2024-10-13 03:07:10


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.825,0.762741,0.731581,0.41276
prob_up,0.0,0.097239,0.093007,0.214855
prob_static,0.875,0.830529,0.808272,0.580543
prob_down,0.125,0.072232,0.098721,0.204602
precision,0.768707,0.77551,0.762821,0.608696
recall,0.719745,0.726115,0.757962,0.713376
f1,0.743421,0.75,0.760383,0.656891
support,"[159.0, 157.0, 159.0]","[159.0, 157.0, 159.0]","[159.0, 157.0, 159.0]","[159.0, 157.0, 159.0]"



AAPL 30m Interval Timestamp: 2024-10-13 03:07:13


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,1.0,0.824227,0.878083,0.645323
prob_up,0.0,0.001264,0.006842,0.054539
prob_static,1.0,0.874448,0.912917,0.746659
prob_down,0.0,0.124288,0.080241,0.198802
precision,0.769373,0.766234,0.740283,0.624413
recall,0.803468,0.795761,0.807322,0.768786
f1,0.786051,0.780718,0.77235,0.689119
support,"[516.0, 519.0, 516.0]","[516.0, 519.0, 516.0]","[516.0, 519.0, 516.0]","[516.0, 519.0, 516.0]"



AAPL 1d Interval Timestamp: 2024-10-13 03:07:21


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,down,down
kelly_1:2.5,0.125,0.369172,0.413283,0.28195
prob_up,0.375,0.549409,0.373657,0.331529
prob_static,0.375,0.025049,0.045426,0.181364
prob_down,0.25,0.425542,0.580917,0.487107
precision,0.672,0.790909,0.690566,0.676471
recall,0.672,0.704453,0.743902,0.560976
f1,0.672,0.745182,0.716243,0.613333
support,"[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]"



AAPL 1wk Interval Timestamp: 2024-10-13 03:07:25


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,down,down
kelly_1:2.5,0.3,0.311302,0.454271,0.324057
prob_up,0.25,0.036117,0.042761,0.136397
prob_static,0.5,0.508073,0.347045,0.34642
prob_down,0.25,0.45581,0.610193,0.517183
precision,0.666667,0.702128,0.734694,0.634146
recall,0.625,0.6875,0.734694,0.530612
f1,0.645161,0.694737,0.734694,0.577778
support,"[48.0, 48.0, 49.0]","[48.0, 48.0, 49.0]","[48.0, 48.0, 49.0]","[48.0, 48.0, 49.0]"


## Sanity Checks

In [79]:
df = f.load('TSLA','5m')
df.tail(3)

,open,high,low,close,adj_close,volume,dividends,stock_splits
Datetime,,,,,,,,
2024-10-11 19:45:00-04:00,218.1000,218.11,218.00,218.0105,218.0105,0,0.0,0.0
2024-10-11 19:50:00-04:00,218.0199,218.05,218.01,218.0200,218.0200,0,0.0,0.0
2024-10-11 19:55:00-04:00,218.0300,218.03,217.98,217.9850,217.9850,0,0.0,0.0


### Check Summary Output

In [47]:
symbol='NVDA'
interval='5m'
period='1mo'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval, period)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

DL Time: 2024-10-10 15:51:25


,model,prediction,kelly_1:2.5,prob_up,prob_static,prob_down,precision,recall,f1,support
0,XGBoost,up,0.475000,0.625000,0.125000,0.250000,0.789091,0.791971,0.790528,"[274.0, 231.0, 285.0]"
1,GradientBoosting,up,0.375125,0.553661,0.071753,0.374586,0.757143,0.773723,0.765343,"[274.0, 231.0, 285.0]"
2,RandomForest,up,0.455799,0.611285,0.076480,0.312235,0.740741,0.729927,0.735294,"[274.0, 231.0, 285.0]"
3,KNN,up,0.347789,0.534135,0.148656,0.317209,0.658915,0.620438,0.639098,"[274.0, 231.0, 285.0]"


In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')